In [17]:
import sympy as sp
import networkx as nx
import numpy as np
from math import factorial
import matplotlib.pyplot as plt
import tqdm

# import linearframework.graph_operations as g_ops
import linearframework.ca_recurrence as ca
import linearframework.linear_framework_results as lfr
import linearframework.gen_graphs as gen
from linearframework.linear_framework_graph import LinearFrameworkGraph

## Implementing LinearFrameworkGraph class

input: 
- list of edges
generates:
- [x] edge_to_sym dictionary
- [x] symbolic laplacian from  edge_to_sym
- [x] node list
- [x] terminal node list
- [ ] nx.digraph of graph (unweighted)

In [18]:
k3_2t_edges = [
    ('1', '2'),
    ('1', '3'),
    ('2', '1'),
    ('2', '3'),
    ('3', '1'),
    ('3', '2'),
    # ('2', '4'),
    # ('3', '5')
]

k3_2t = LinearFrameworkGraph(k3_2t_edges)

In [19]:
def generalized_randomness_parameter(graph, source, target, moment):
    """calculates the symbolic expression of the moment-th moment of the graph over the mean of the graph to the power of moment

    Args:
        edge_to_weight (dict[tuple[str]: float]): dict of edges in form {('v_1', 'v_2): w} where w is some positive number
        edge_to_sym (dict[tuple[str]: sp.core.symbol.Symbol]): dict of edges in form {('v_1', 'v_2): l} where l is some sympy symbol
        source (str): id of source vertex
        target (str): id of target vertex
        moment (int): moment of interest

    Returns:
        sympy.core.add.Add: symbolic expression of the moment-th moment of the graph over the mean of the graph to the power of moment
    """
    if not isinstance(graph, LinearFrameworkGraph):
        raise NotImplementedError("graph must be a LinearFrameworkGraph with no more than one terminal vertex")
    if len(graph.terminal_nodes) > 1:
        raise NotImplementedError("graph must be a LinearFrameworkGraph with no more than one terminal vertex")

    if not isinstance(source, str) or source not in list(graph.nodes):
        raise NotImplementedError("source must be a string and must be the id of a vertex in graph")
    if not isinstance(target, str) or target not in list(graph.nodes):
        raise NotImplementedError("target must be a string and must be the id of a vertex in graph")
    if not isinstance(moment, int) or moment <= 0:
        raise NotImplementedError("moment must be a natural number")
    
    sym_lap = graph.sym_lap
    n = len(graph.nodes)
    Q_n_minus_2 = ca.get_sigma_Q_k(sym_lap, n-2)[1]
    
    numerator = lfr._ca_kth_moment_numerator(graph, Q_n_minus_2, source, target, moment)
    denominator = lfr._ca_kth_moment_numerator(graph, Q_n_minus_2, source, target, 1) ** moment
    return numerator / denominator

In [20]:
n = 2
erlang_edges = list(gen.gen_erlang_process_dict(n, rate=10).keys())
erlang = LinearFrameworkGraph(erlang_edges)
sym_generalized_randomness_param = generalized_randomness_parameter(erlang, '1', '2', 2)

sym_generalized_randomness_param

2

In [21]:
free_symbols = sym_generalized_randomness_param.free_symbols

e_r = sp.symbols('e_r')

symbol_dict = {}
for free_symbol in free_symbols:
    symbol_dict[free_symbol] = e_r

generalized_randomness_param = float(sym_generalized_randomness_param.subs(symbol_dict))

generalized_randomness_param

2.0

In [22]:
gen.gen_erlang_process_dict(3, rate=10)

{('1', '2'): 10, ('2', '3'): 10}